In [10]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수를 로드합니다.
load_dotenv()

# API 키가 제대로 로드되었는지 확인하는 코드 (선택 사항이지만 추천)
if not os.environ.get("GROQ_API_KEY"):
    raise ValueError("GROQ_API_KEY가 .env 파일에 없거나 로드되지 않았습니다.")

In [3]:
# === 1. 라이브러리 임포트 및 설정 (Vector DB 관련 추가) ===
import os
import json
from dotenv import load_dotenv
from typing import List
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings # 임베딩을 위해 추가
from langchain_community.vectorstores import FAISS # Vector DB로 FAISS 추가
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, BaseMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END, MessagesState
from langgraph.prebuilt import ToolNode

# --- .env 파일 로드 ---
load_dotenv()
# Vector DB에서 사용할 OpenAI API 키도 확인
for key in ["GROQ_API_KEY", "OPENAI_API_KEY"]:
    if not os.environ.get(key):
        raise ValueError(f"{key}가 .env 파일에 없거나 로드되지 않았습니다.")

# === 2. 파일 데이터 로드 및 Vector DB 초기화 (수정된 부분) ===
def parse_menu_file(file_path: str) -> dict:
    # (이전과 동일한 커스텀 파서)
    menu_db = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line or ':' not in line: continue
            menu_name, details = [part.strip() for part in line.split(':', 1)]
            if '₩' in details:
                description, price = [part.strip() for part in details.rsplit('₩', 1)]
                price = '₩' + price
            else:
                description = details
                price = "가격 정보 없음"
            menu_db[menu_name] = {"가격": price, "설명": description}
    return menu_db

try:
    cafe_menu_db = parse_menu_file('../data/cafe_menu_data.txt')
    print("✅ 카페 메뉴 데이터를 파일에서 성공적으로 불러왔습니다.")
except FileNotFoundError:
    raise ValueError("`../data/cafe_menu_data.txt` 파일을 찾을 수 없습니다.")

# [새로운 부분] Vector DB 인덱스 생성
documents = []
for menu_name, details in cafe_menu_db.items():
    # Vector DB에 저장할 Document 객체 생성
    content = f"메뉴명: {menu_name}\n가격: {details['가격']}\n설명: {details['설명']}"
    doc = Document(page_content=content, metadata={"menu_name": menu_name})
    documents.append(doc)

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings()

# FAISS Vector Store에 documents를 임베딩하여 저장
vector_store = FAISS.from_documents(documents, embeddings)

# retriever 생성 (유사도 높은 상위 2개 결과를 가져오도록 설정)
retriever = vector_store.as_retriever(search_kwargs={"k": 2})
print("✅ FAISS Vector DB 인덱싱 및 Retriever 생성이 완료되었습니다.")


# === 3. 카페 메뉴 검색 도구 수정 (Vector DB Retriever 사용) ===
@tool
def search_cafe_menu(query: str) -> str:
    """사용자가 질문한 카페 메뉴(음료 또는 디저트)와 가장 관련성이 높은 메뉴 정보를 Vector DB에서 찾습니다."""
    print(f"--- Vector DB 검색 실행: retriever.invoke(query='{query}') ---")
    
    # retriever를 사용하여 의미적으로 유사한 문서를 검색
    retrieved_docs = retriever.invoke(query)
    
    if not retrieved_docs:
        return f"'{query}'와 관련된 메뉴를 찾을 수 없습니다."
    
    # 검색된 결과를 하나의 문자열로 정리하여 반환
    return "\n\n".join([doc.page_content for doc in retrieved_docs])


# === 4. ~ 8. 나머지 Agent 로직은 이전과 완전히 동일합니다. ===
# (Agent는 도구의 내부 구현이 어떻게 바뀌었는지 전혀 신경쓰지 않습니다.)

class AgentState(MessagesState):
    pass

tools = [search_cafe_menu]
llm = ChatGroq(model_name="llama3-8b-8192")
system_prompt = (
    "당신은 '카페봇'이라는 이름을 가진 친절한 카페 직원 Agent입니다. "
    "사용자의 질문에 답변하기 위해 `search_cafe_menu` 도구를 사용해야 합니다. "
    "도구 사용 후에는, 그 결과를 바탕으로 사용자에게 자연스러운 문장으로 답변을 정리해서 제공해야 합니다. "
    "--- \n"
    "**매우 중요: 사용자의 모든 질문에 대해 반드시 한국어로만 답변해야 합니다.**"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
agent_runnable = prompt | llm.bind_tools(tools)
def call_agent_node(state: AgentState):
    print("--- Agent 노드 실행 ---")
    response = agent_runnable.invoke(state)
    return {"messages": [response]}
tool_node = ToolNode(tools)
def should_continue(state: AgentState) -> str:
    if state["messages"][-1].tool_calls:
        return "tools"
    return "final_answer"
def format_final_response(state: AgentState):
    print("--- 최종 답변 생성 ---")
    final_answer: AIMessage = state["messages"][-1]
    clean_content = final_answer.content.replace('<tool-use>{"tool_calls":[]}</tool-use>', '').strip()
    if clean_content:
        print("🤖 카페봇의 답변:\n\n" + clean_content)
    else:
        print("🤖 카페봇: 더 이상 드릴 정보가 없습니다.")
    return {"messages": [final_answer]}
builder = StateGraph(AgentState)
builder.add_node("agent", call_agent_node)
builder.add_node("tools", tool_node)
builder.add_node("final_answer", format_final_response)
builder.set_entry_point("agent")
builder.add_conditional_edges("agent", should_continue)
builder.add_edge("tools", "agent")
builder.add_edge("final_answer", END)
graph = builder.compile()
def run_agent(question: str):
    print(f"\n===== 사용자 질문: {question} =====\n")
    inputs = {"messages": [HumanMessage(content=question)]}
    graph.invoke(inputs, {"recursion_limit": 10})

try:
    run_agent("달달하고 부드러운 커피 뭐 있어?")
    run_agent("티라미수 케이크에 대해 알려줘")
except Exception as e:
    print(f"\n오류가 발생했습니다: {e}")

✅ 카페 메뉴 데이터를 파일에서 성공적으로 불러왔습니다.
✅ FAISS Vector DB 인덱싱 및 Retriever 생성이 완료되었습니다.

===== 사용자 질문: 달달하고 부드러운 커피 뭐 있어? =====

--- Agent 노드 실행 ---
--- Vector DB 검색 실행: retriever.invoke(query='달달하고 부드러운 커피') ---
--- Agent 노드 실행 ---
--- 최종 답변 생성 ---
🤖 카페봇의 답변:

달달하고 부드러운 커피를 찾으신 것은요! 카페 라떼가 좋은 선택이 될 듯합니다. 부드러운 우유와 에스프레소의 조화를 잘 느끼실 수 있습니다. 가격은 ₩4,500입니다.

===== 사용자 질문: 티라미수 케이크에 대해 알려줘 =====

--- Agent 노드 실행 ---
--- Vector DB 검색 실행: retriever.invoke(query='티라미수 케이크') ---
--- Agent 노드 실행 ---
--- 최종 답변 생성 ---
🤖 카페봇의 답변:

티라미수 케이크는 찾을 수 없습니다. 하지만, 유사한 메뉴가 있습니다.チ즈 케이크는 부드럽고 진한 치즈의 맛을 즐길 수 있습니다. 초콜릿 라떼는 진한 초콜릿과 우유가 만난 음료입니다. Would you like to try one of these options?
